In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import os
import re
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from pathlib import Path  # Python 3.6+ only
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

In [ ]:
#### Sample playlist
kittinan_links = 'https://gist.githubusercontent.com/kittinan/36b9977d9273755f9ca953cae31419d3/raw/50383e756bc34cfede4c58018f909805ecd1cfbc/my-playlist.json'
dct_kittinan_playlist = requests.get(kittinan_links).json()

In [ ]:
#### Spotify Authentication
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=os.getenv('SPOTIFY_CLIENT_ID'),
                                                           client_secret=os.getenv('SPOTIFY_SECRET')))

In [ ]:
playlists = sp.user_playlists('spotify')
lst_playlist = []
while playlists:
    lst_playlist += [playlist for playlist in playlists['items']]
    playlists = sp.next(playlists) if playlists['next'] else None
    
df_spotify_playlist = pd.DataFrame(lst_playlist)

In [ ]:
results = sp.search(q='paul mccartney', limit=20)
df_spotify_query = pd.DataFrame(results['tracks']['items'])